In [1]:
from genfft import generate_fft, default_config, fft_two_factor_twiddle
import numpy as np
from copy import copy

In [2]:
cfg = copy(default_config)
cfg["generator_path"] = "../genfft"
cfg["compile_args"] = ["-x", "c++", "-O3", "-include", "genfft/codelet.hh", "-shared"]

In [3]:
fft12 = fft_two_factor_twiddle(cfg, 3, 4)

x = np.arange(12, dtype='complex64')

fft12(x) - np.fft.fft(x)

array([ 0.0000000e+00+0.00000000e+00j,  8.8817842e-16-1.37861639e-06j,
        0.0000000e+00-4.24942073e-07j,  0.0000000e+00+0.00000000e+00j,
        0.0000000e+00-5.39011657e-07j, -8.8817842e-16+4.24942073e-07j,
        0.0000000e+00+0.00000000e+00j, -8.8817842e-16-4.24942073e-07j,
        0.0000000e+00+5.39011657e-07j,  0.0000000e+00+0.00000000e+00j,
        0.0000000e+00+4.24942073e-07j,  8.8817842e-16+1.37861639e-06j])

In [13]:
fft27 = fft_two_factor_twiddle(cfg, 3, 9)
x = np.arange(27, dtype='complex64')
np.abs(fft27(x) - np.fft.fft(x)).max()

8.962143078707292e-06

In [14]:
"onuth".splitlines()

['onuth']

In [17]:
x = np.arange(10, dtype='complex64')

In [18]:
x.real

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)

In [19]:
x.real[3] = 42

In [20]:
x

array([ 0.+0.j,  1.+0.j,  2.+0.j, 42.+0.j,  4.+0.j,  5.+0.j,  6.+0.j,
        7.+0.j,  8.+0.j,  9.+0.j], dtype=complex64)

In [4]:
fft12 = generate_fft(cfg, "notw_complex", n=12)
x = np.arange(12, dtype='complex64')
y = np.zeros_like(x)
fft12(x, y) 
y - np.fft.fft(x)

array([ 0.0000000e+00+0.00000000e+00j,  8.8817842e-16-1.37861639e-06j,
        0.0000000e+00-4.24942073e-07j,  0.0000000e+00+0.00000000e+00j,
        0.0000000e+00-6.21744984e-08j, -8.8817842e-16+4.24942073e-07j,
        0.0000000e+00+0.00000000e+00j, -8.8817842e-16-4.24942073e-07j,
        0.0000000e+00+6.21744984e-08j,  0.0000000e+00+0.00000000e+00j,
        0.0000000e+00+4.24942073e-07j,  8.8817842e-16+1.37861639e-06j])

In [8]:
y

array([ 30.       +0.j,  36.       +0.j,  17.196152 +0.j, -30.320509 +0.j,
       -16.392305 +0.j,   4.392305 +0.j,  11.       +0.j,  -1.       +0.j,
        -9.464102 +0.j,  -2.5358987+0.j,   6.8038483+0.j,   4.3205085+0.j],
      dtype=complex64)